# Image Detection Demo with PytorchWildlife

This tutorial guides you on how to use PyTorchWildlife to separate positive and negative animal detections. We will go through the process of setting up the environment, defining the detection model, as well as performing inference and saving the results in different ways.

## Prerequisites
Install PytorchWildlife running the following commands:
```bash
conda create -n pytorch_wildlife python=3.8 -y
conda activate pytorch_wildlife
pip install PytorchWildlife
```
Also, make sure you have a CUDA-capable GPU if you intend to run the model on a GPU. This notebook can also run on CPU.

## Importing libraries
First, we'll start by importing the necessary libraries and modules.

In [2]:
import sys
import os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), "..")))

In [3]:
import torch
from PytorchWildlife.models import detection as pw_detection
from PytorchWildlife import utils as pw_utils

## Model Initialization
We will initialize the MegaDetectorV5 model for image detection. This model is designed for detecting animals in images.

In [5]:
# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available!")
    # Get the number of available devices
    num_devices = torch.cuda.device_count()
    print(f"Number of CUDA devices: {num_devices}")
    
    # List all devices and their names
    for i in range(num_devices):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
    
    # Optionally, set a specific device as the active device
    torch.cuda.set_device(0)  # Switch to device 1
    print(f"Current CUDA device set to: {torch.cuda.current_device()}")

else:
    print("CUDA is not available.")


CUDA is available!
Number of CUDA devices: 2
Device 0: NVIDIA GeForce RTX 4090
Device 1: NVIDIA GeForce RTX 4090
Current CUDA device set to: 0


In [13]:
# Setting the device to use for computations ('cuda' indicates GPU)
#DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
#if DEVICE == "cuda":
#    torch.cuda.set_device(0)

# Initializing the MegaDetectorV6 model for image detection
#detection_model = pw_detection.MegaDetectorV6(device=DEVICE, pretrained=True, version="yolov9c")


# Initialize the MegaDetectorV6 model without specifying the device
detection_model = pw_detection.MegaDetectorV6(pretrained=True, version="yolov9c")

# Move the model explicitly to cuda:1
detection_model = detection_model.to("cuda:1")


 
# Uncomment the following line to use MegaDetectorV5 instead of MegaDetectorV6
#detection_model = pw_detection.MegaDetectorV5(device=DEVICE, pretrained=True, version="a")


Ultralytics 8.3.28 🚀 Python-3.8.20 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
YOLOv9c summary (fused): 384 layers, 25,321,561 parameters, 0 gradients, 102.3 GFLOPs


## Variable definition
In order to process the batch detection, we will define an input directory where the images are stored, a confidence threshold and an output directory to copy the positive and negative images into distinctive folders. If you want to follow this tutorial with your own data, modify the following variables.

In [14]:
#tgt_folder_path = os.path.join(".","demo_data","imgs")
tgt_folder_path = "/media/mo/nvme0n1/MDv5_v6_test_HG"

#output_path = "folder_separation"
output_path = "/media/mo/nvme0n1/MDv5_test_HG_separated"
#output_path = "/media/mo/nvme0n1/MDv6_test_HG_separated"
#output_path = "/media/mo/nvme0n1/MDv6_test_HG_separated_subfolder"
threshold = 0.2




## Batch Image Detection
Next, we'll demonstrate how to process multiple images in batches. This is useful when you have a large number of images and want to process them efficiently.

In [15]:
results = detection_model.batch_image_detection(tgt_folder_path, batch_size=100) #default batch_size=16, highVRAM,failed= 350

  0%|          | 0/1929 [00:00<?, ?it/s]

  0%|          | 0/1929 [00:10<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.46 GiB. GPU 0 has a total capacity of 23.65 GiB of which 738.81 MiB is free. Process 13848 has 1.37 GiB memory in use. Process 701580 has 736.00 MiB memory in use. Process 853442 has 19.44 GiB memory in use. Including non-PyTorch memory, this process has 1.36 GiB memory in use. Of the allocated memory 891.77 MiB is allocated by PyTorch, and 16.23 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

## Separate positive and negative detections
PytorchWildlife allows to copy the files from your original folder to a new directory containing the "Animal" and "No-animal" subdirectories. A detection is considered positive if the prediction confidence is higher than the threshold

In [9]:
os.makedirs(output_path, exist_ok=True)
json_file = os.path.join(output_path, "detection_results.json")
pw_utils.save_detection_json(results, json_file,
                             categories=detection_model.CLASS_NAMES,
                             exclude_category_ids=[], # Category IDs can be found in the definition of each model.
                             exclude_file_path=tgt_folder_path)

# Separate the positive and negative detections through file copying:
pw_utils.detection_folder_separation(json_file, tgt_folder_path, output_path, threshold)

'81353 files were successfully separated'

### Copyright (c) Microsoft Corporation. All rights reserved.
### Licensed under the MIT License.